<a href="https://colab.research.google.com/github/alex-movila/ML-Colab-Tutorials/blob/master/GNN_based_Screening_Predictor%3F!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ogb
!pip install optax

In [ ]:
from ogb.graphproppred import GraphPropPredDataset

dataset = GraphPropPredDataset(name='ogbg-molhiv')

graph, label = dataset[0]
print('Keys are', graph.keys())
print('Label is', label)

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:00<00:00,  6.70it/s]


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:00<00:00, 81409.03it/s]


Saving...
Keys are dict_keys(['edge_index', 'edge_feat', 'node_feat', 'num_nodes'])
Label is [0]


In [ ]:
print(graph['num_nodes'])
print(graph['node_feat'].shape)
print(graph['edge_feat'].shape)
print(graph['edge_index'].shape)

19
(19, 9)
(40, 3)
(2, 40)


In [ ]:
print(graph['node_feat'][:5, :])
print(graph['edge_feat'][:5, :])

[[ 5  0  4  5  3  0  2  0  0]
 [ 5  0  4  5  2  0  2  0  0]
 [ 5  0  3  5  0  0  1  0  1]
 [ 7  0  2  6  0  0  1  0  1]
 [28  0  4  2  0  0  5  0  1]]
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 0 0]]


In [ ]:
print(graph['edge_index'])

[[ 0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  6  9  4 10 10 11 11 12
  12 13 11 14 14 15 15 16 16 17 15 18  9  2 18  4]
 [ 1  0  2  1  3  2  4  3  5  4  6  5  7  6  8  7  9  6 10  4 11 10 12 11
  13 12 14 11 15 14 16 15 17 16 18 15  2  9  4 18]]


In [ ]:
import numpy as np

def convert_edge_index_to_matrix(edge_index, nb_nodes):
  adj_mat = np.eye(nb_nodes)
  for i in range(edge_index.shape[1]):
    adj_mat[edge_index[0, i], edge_index[1, i]] = 1.0
  return adj_mat / np.sum(adj_mat, axis=-1, keepdims=True)

print(convert_edge_index_to_matrix(graph['edge_index'], graph['num_nodes']))

In [ ]:
import jax
import jax.numpy as jnp

@jax.jit
def simple_gnn_layer(weights, features, adj_matrix):
  latent = jnp.matmul(features, weights)  # (N x H) * (H x H') -> (N x H')
  latent = jnp.matmul(adj_matrix, latent)  # (N x N) * (N x H') -> (N x H')
  latent = jax.nn.relu(latent)
  return latent

@jax.jit
def network(params, features, adj_matrix):
  latent = features
  for layer in range(len(params) - 1):
    latent = simple_gnn_layer(params[layer], latent, adj_matrix) # (N x H)
  g_features = jnp.mean(latent, axis=0)  # (H,)
  logits = jnp.matmul(g_features, params[-1])  # (1,)
  return logits

In [ ]:
@jax.jit
def binary_cross_entropy(logits, labels):
  max_val = jnp.clip(logits, 0, None)
  loss = logits - logits * labels + max_val + jnp.log(
      jnp.exp(-max_val) + jnp.exp((-logits - max_val)))
  return jnp.mean(loss)

@jax.jit
def _loss(params, features, adj_matrix, labels):
  logits = network(params, features, adj_matrix)
  return binary_cross_entropy(logits, labels)

@jax.jit
def accuracy(logits, labels):
  return jnp.mean((logits > 0.0) == (labels > 0.5))

In [ ]:
import optax

split_idx = dataset.get_idx_split()
train_idx, val_idx = split_idx["train"], split_idx["valid"]
input_dim = graph['node_feat'].shape[1]

def train(hidden_dim, nb_layers, epochs, learning_rate):
  params = []
  params.append(np.random.randn(input_dim, hidden_dim) / np.sqrt(input_dim))
  for i in range(nb_layers - 2):
    params.append(np.random.randn(hidden_dim, hidden_dim) / np.sqrt(hidden_dim))
  params.append(np.random.randn(hidden_dim, 1) / np.sqrt(hidden_dim))

  opt = optax.adam(learning_rate=learning_rate)
  opt_state = opt.init(params)

  @jax.jit
  def _step(params, opt_state, features, adj_matrix, labels):
    loss, grads = jax.value_and_grad(_loss)(
        params, features, adj_matrix, labels)
    updates, opt_state = opt.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss
  
  ep = 0
  step = 0
  while ep < epochs:
    for idx in train_idx:
      graph, label = dataset[idx]
      node_fts = graph['node_feat']
      nb_nodes = graph['num_nodes']
      adj_mat = convert_edge_index_to_matrix(graph['edge_index'], nb_nodes)
      params, opt_state, loss = _step(
          params, opt_state, node_fts, adj_mat, label
      )
      if step % 1000 == 0:
        print(f'step = {step} | loss = {loss}')
      step += 1
    val_preds = []
    val_labels = []
    for idx in val_idx:
      graph, label = dataset[idx]
      node_fts = graph['node_feat']
      nb_nodes = graph['num_nodes']
      adj_mat = convert_edge_index_to_matrix(graph['edge_index'], nb_nodes)
      val_preds.append(network(params, node_fts, adj_mat)[0])
      val_labels.append(label)
    val_accuracy = accuracy(jnp.array(val_preds), jnp.array(val_labels))
    print(f'epoch = {ep} | validation accuracy = {val_accuracy}')
    ep += 1
  return params

In [ ]:
trained_model = train(hidden_dim=32, nb_layers=2, epochs=1, learning_rate=0.001)

step = 0 | loss = 3.228520154953003
step = 1000 | loss = 0.00039005279541015625
step = 2000 | loss = 1.0528050661087036
step = 3000 | loss = 0.017929553985595703
step = 4000 | loss = 0.02083730697631836
step = 5000 | loss = 0.016472816467285156
step = 6000 | loss = 0.010475635528564453
step = 7000 | loss = 0.02108168601989746
step = 8000 | loss = 0.0829765796661377
step = 9000 | loss = 0.002238750457763672
step = 10000 | loss = 0.012306690216064453
step = 11000 | loss = 0.0037064552307128906
step = 12000 | loss = 0.026071548461914062
step = 13000 | loss = 0.012278556823730469
step = 14000 | loss = 0.02101278305053711
step = 15000 | loss = 0.025160551071166992
step = 16000 | loss = 0.029111146926879883
step = 17000 | loss = 0.07236838340759277
step = 18000 | loss = 0.02436351776123047
step = 19000 | loss = 0.018687963485717773
step = 20000 | loss = 0.06229758262634277
step = 21000 | loss = 0.013743877410888672
step = 22000 | loss = 0.018528223037719727
step = 23000 | loss = 0.0512700080